In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/ln/h2lpkllx5fl21m8307tnfsdc0000gn/T/ipykernel_15093/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [5]:
# Your code
from sklearn.model_selection import train_test_split

# Definir las características (X) y las etiquetas (y)
X = data['text']  # Cambia 'text' por el nombre correcto de la columna con texto
y = data['label']  # Cambia 'label' por el nombre de la columna con etiquetas

# Dividir los datos en entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar las formas de los conjuntos divididos
print("Tamaño de X_train:", X_train.shape)
print("Tamaño de X_test:", X_test.shape)
print("Tamaño de y_train:", y_train.shape)
print("Tamaño de y_test:", y_test.shape)


Tamaño de X_train: (800,)
Tamaño de X_test: (200,)
Tamaño de y_train: (800,)
Tamaño de y_test: (200,)


## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [8]:
# Your code

def remove_inline_js_css(text):
    text = re.sub(r'<(script|style).*?>.*?(<\/\1>)', '', text, flags=re.S)
    return text

def remove_html_comments(text):
    text = re.sub(r'<!--.*?-->', '', text, flags=re.S)
    return text

def remove_html_tags(text):
    text = re.sub(r'<[^>]+>', '', text)
    return text

def clean_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Eliminar stopwords
    stop_words = set(stopwords.words('english'))
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text



In [9]:
def stem_text(text):
    stemmer = SnowballStemmer('english')
    text = " ".join([stemmer.stem(word) for word in text.split()])
    return text


In [11]:
def preprocess_text(text):
    text = remove_inline_js_css(text)
    text = remove_html_comments(text)
    text = remove_html_tags(text)
    text = clean_text(text)
    text = stem_text(text)
    return text




In [13]:
import re


In [14]:


# Aplicar al dataset
data['cleaned_text'] = data['text'].apply(preprocess_text)


In [15]:
print(data[['text', 'cleaned_text']].head())


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  dear sir strict privat busi propos mike chukwu...  
1                                                     
2  noracheryl email dozen memo haiti weekend plea...  
3  dear sir2fmadam2c know propos might surpris em...  
4                                                fyi  


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [16]:
# Your code
def additional_cleaning_steps(text):
    # Eliminar caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Mantener solo letras y espacios
    # Eliminar números
    text = re.sub(r'\d+', '', text)
    # Eliminar caracteres únicos
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    # Eliminar caracteres únicos al inicio de las palabras
    text = re.sub(r'\b[a-zA-Z]\s', '', text)
    # Sustituir múltiples espacios por un único espacio
    text = re.sub(r'\s+', ' ', text)
    # Eliminar prefijos 'b' (por ejemplo, en cadenas bytes como b'texto')
    text = re.sub(r'^b\'|^b\"', '', text)
    # Convertir a minúsculas
    text = text.lower()
    return text


In [17]:
# Modificar el pipeline para incluir pasos adicionales
def preprocess_text_full(text):
    text = remove_inline_js_css(text)
    text = remove_html_comments(text)
    text = remove_html_tags(text)
    text = clean_text(text)
    text = stem_text(text)
    text = additional_cleaning_steps(text)  # Incluir pasos adicionales
    return text

# Aplicar al dataset
data['cleaned_text'] = data['text'].apply(preprocess_text_full)


In [18]:
print(data[['text', 'cleaned_text']].head())


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  dear sir strict privat busi propos mike chukwu...  
1                                                     
2  noracheryl email dozen memo haiti weekend plea...  
3  dear sirfmadamc know propos might surpris emer...  
4                                                fyi  


## Now let's work on removing stopwords
Remove the stopwords.

In [19]:
# Your code
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kikegarciabello/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords

# Cargar la lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    # Dividir el texto en palabras, eliminar stopwords, y volver a unir el texto
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text


In [21]:
def preprocess_text_full(text):
    text = remove_inline_js_css(text)
    text = remove_html_comments(text)
    text = remove_html_tags(text)
    text = clean_text(text)
    text = additional_cleaning_steps(text)
    text = remove_stopwords(text)  # Eliminar stopwords
    return text

# Aplicar al dataset
data['cleaned_text'] = data['text'].apply(preprocess_text_full)


In [22]:
print(data[['text', 'cleaned_text']].head())


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  dear sir strictly private business proposal mi...  
1                                                     
2  noracheryl emailed dozens memos haiti weekend ...  
3  dear sirfmadamc know proposal might surprise e...  
4                                                fyi  


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [23]:
# Your code
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Opcional, mejora la lemmatización en algunos casos


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kikegarciabello/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/kikegarciabello/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [24]:
from nltk.stem import WordNetLemmatizer

# Crear un lematizador
lemmatizer = WordNetLemmatizer()


In [25]:
def lemmatize_text(text):
    # Dividir el texto en palabras y aplicar lemmatization a cada palabra
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    return text


In [26]:
def preprocess_text_full(text):
    text = remove_inline_js_css(text)
    text = remove_html_comments(text)
    text = remove_html_tags(text)
    text = clean_text(text)
    text = additional_cleaning_steps(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)  # Aplicar lemmatization
    return text

# Aplicar al dataset
data['cleaned_text'] = data['text'].apply(preprocess_text_full)


In [27]:
print(data[['text', 'cleaned_text']].head())


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                        cleaned_text  
0  dear sir strictly private business proposal mi...  
1                                                     
2  noracheryl emailed dozen memo haiti weekend pl...  
3  dear sirfmadamc know proposal might surprise e...  
4                                                fyi  


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [40]:
print("Número de mensajes originales:")
print(data['text'].str.strip().replace("", float("NaN")).dropna().shape[0])

print("\nPrimeros mensajes originales:")
print(data['text'].head())


Número de mensajes originales:
983

Primeros mensajes originales:
0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
2    Nora--Cheryl has emailed dozens of memos about...
3    Dear Sir=2FMadam=2C I know that this proposal ...
4                                                  fyi
5    sure -- bottom line - you need a special secur...
Name: text, dtype: object


In [41]:
def additional_cleaning_steps(text):
    # Mantener palabras y espacios básicos (evitar eliminación excesiva)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Permitir números por ahora
    text = re.sub(r'\s+', ' ', text)  # Reemplazar múltiples espacios por uno
    return text


In [42]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.difference_update(['not', 'no', 'will', 'do'])  # Mantener palabras clave


In [43]:
def preprocess_text_minimal(text):
    text = additional_cleaning_steps(text)
    # No eliminar stopwords ni realizar lematización por ahora
    return text

# Aplicar al dataset
data['cleaned_text'] = data['text'].apply(preprocess_text_minimal)

# Filtrar mensajes vacíos después del preprocesamiento
data = data[data['cleaned_text'].str.strip() != ""]

# Separar mensajes HAM y SPAM
ham_messages = data[data['label'] == 'ham']['cleaned_text']
spam_messages = data[data['label'] == 'spam']['cleaned_text']

print("Número de mensajes HAM:", len(ham_messages))
print("Número de mensajes SPAM:", len(spam_messages))


Número de mensajes HAM: 0
Número de mensajes SPAM: 0


In [44]:
print("Número de filas en el archivo original:", data.shape[0])
print("Columnas disponibles:", data.columns)

# Revisar algunos ejemplos del archivo original
print(data.head(10))


Número de filas en el archivo original: 983
Columnas disponibles: Index(['text', 'label', 'cleaned_text'], dtype='object')
                                                 text  label  \
0   DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1   
2   Nora--Cheryl has emailed dozens of memos about...      0   
3   Dear Sir=2FMadam=2C I know that this proposal ...      1   
4                                                 fyi      0   
5   sure -- bottom line - you need a special secur...      0   
6   Dear Sir,I am Engr. Ugo Nzego with the Enginee...      1   
7   Abedin Huma <AbedinH@state.gov>Saturday Novemb...      0   
8   There is an Oct 16th George Marshall event at ...      0   
9   <P>1 25% for you as the account owner <BR>2 65...      1   
10  STRONG><A href=3D"http://www.cnn.com/2003/WORL...      1   

                                         cleaned_text  
0   DEAR SIR STRICTLY A PRIVATE BUSINESS PROPOSAL ...  
2   NoraCheryl has emailed dozens of memos about H...  
3   

In [45]:
def additional_cleaning_steps(text):
    # Mantener letras, números y espacios (evitar eliminar demasiado contenido)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Elimina solo caracteres especiales
    text = re.sub(r'\s+', ' ', text)  # Reemplaza múltiples espacios por uno
    return text


In [46]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.difference_update(['do', 'will', 'fyi'])  # Permitir palabras clave


In [47]:
def preprocess_text_minimal(text):
    text = text.lower()  # Convertir a minúsculas
    text = additional_cleaning_steps(text)  # Limpiar caracteres
    words = text.split()
    # Eliminar stopwords personalizadas
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

# Aplicar al dataset
data['cleaned_text'] = data['text'].apply(preprocess_text_minimal)

# Filtrar mensajes vacíos después del preprocesamiento
data = data[data['cleaned_text'].str.strip() != ""]

# Separar mensajes HAM y SPAM
ham_messages = data[data['label'] == 0]['cleaned_text']
spam_messages = data[data['label'] == 1]['cleaned_text']

print("Número de mensajes HAM:", len(ham_messages))
print("Número de mensajes SPAM:", len(spam_messages))


Número de mensajes HAM: 544
Número de mensajes SPAM: 439


In [48]:
print("Ejemplo de mensajes HAM procesados:")
print(ham_messages.head())

print("\nEjemplo de mensajes SPAM procesados:")
print(spam_messages.head())


Ejemplo de mensajes HAM procesados:
2    noracheryl emailed dozens memos haiti weekend ...
4                                                  fyi
5    sure bottom line need special security code ge...
7    abedin huma abedinhstategovsaturday november 2...
8    oct 16th george marshall event department do t...
Name: cleaned_text, dtype: object

Ejemplo de mensajes SPAM procesados:
0     dear sir strictly private business proposal mi...
3     dear sir2fmadam2c know proposal might surprise...
6     dear siri engr ugo nzego engineering stores de...
9     p1 25 account owner br2 65 colleaguesbr3 10 wi...
10    stronga href3dhttpwwwcnncom2003worldafrica0720...
Name: cleaned_text, dtype: object


In [49]:
from sklearn.feature_extraction.text import CountVectorizer

# Crear un vectorizador
vectorizer = CountVectorizer()

# Transformar mensajes HAM
ham_bow = vectorizer.fit_transform(ham_messages)
ham_words = pd.DataFrame(ham_bow.toarray(), columns=vectorizer.get_feature_names_out())
ham_top_words = ham_words.sum(axis=0).sort_values(ascending=False).head(10)

# Transformar mensajes SPAM
spam_bow = vectorizer.fit_transform(spam_messages)
spam_words = pd.DataFrame(spam_bow.toarray(), columns=vectorizer.get_feature_names_out())
spam_top_words = spam_words.sum(axis=0).sort_values(ascending=False).head(10)

print("Top palabras HAM:")
print(ham_top_words)

print("\nTop palabras SPAM:")
print(spam_top_words)


Top palabras HAM:
will         186
us           114
pm           107
would        106
do            97
president     86
call          75
percent       73
2010          68
work          67
dtype: int64

Top palabras SPAM:
will           1716
money           918
account         736
bank            720
us              475
business        470
fund            426
transaction     404
transfer        390
country         361
dtype: int64


## Extra features

In [51]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
# Definir listas de palabras sospechosas y símbolos de dinero
money_simbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

# Agregar columnas adicionales al DataFrame
data['money_mark'] = data['cleaned_text'].str.contains(money_simbol_list).astype(int)
data['suspicious_words'] = data['cleaned_text'].str.contains(suspicious_words).astype(int)
data['text_len'] = data['cleaned_text'].apply(lambda x: len(x))

print(data.head())


                                                text  label  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1   
2  Nora--Cheryl has emailed dozens of memos about...      0   
3  Dear Sir=2FMadam=2C I know that this proposal ...      1   
4                                                fyi      0   
5  sure -- bottom line - you need a special secur...      0   

                                        cleaned_text  money_mark  \
0  dear sir strictly private business proposal mi...           1   
2  noracheryl emailed dozens memos haiti weekend ...           1   
3  dear sir2fmadam2c know proposal might surprise...           1   
4                                                fyi           1   
5  sure bottom line need special security code ge...           1   

   suspicious_words  text_len  
0                 1      1569  
2                 0       115  
3                 1      1491  
4                 0         3  
5                 0       330  


## How would work the Bag of Words with Count Vectorizer concept?

In [52]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Vectorizar todo el conjunto de datos
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(data['cleaned_text'])

# Ver la forma de la matriz
print("Forma del dataset vectorizado con Bag of Words:", X_bow.shape)


Forma del dataset vectorizado con Bag of Words: (983, 22791)


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [53]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Ver la forma del dataset vectorizado
print("Forma del dataset vectorizado con TF-IDF:", X_tfidf.shape)


Forma del dataset vectorizado con TF-IDF: (983, 22791)


## And the Train a Classifier?

In [54]:
# Your code
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# Dividir el dataset en entrenamiento y prueba
X_train_bow, X_test_bow, y_train, y_test = train_test_split(X_bow, data['label'], test_size=0.2, random_state=42)
X_train_tfidf, X_test_tfidf, _, _ = train_test_split(X_tfidf, data['label'], test_size=0.2, random_state=42)

# Entrenar el modelo con Bag of Words
model_bow = MultinomialNB()
model_bow.fit(X_train_bow, y_train)
y_pred_bow = model_bow.predict(X_test_bow)

# Entrenar el modelo con TF-IDF
model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

# Evaluar el modelo
print("Resultados con Bag of Words:")
print("Accuracy:", accuracy_score(y_test, y_pred_bow))
print(classification_report(y_test, y_pred_bow))

print("\nResultados con TF-IDF:")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


Resultados con Bag of Words:
Accuracy: 0.9035532994923858
              precision    recall  f1-score   support

           0       0.95      0.88      0.91       112
           1       0.85      0.94      0.89        85

    accuracy                           0.90       197
   macro avg       0.90      0.91      0.90       197
weighted avg       0.91      0.90      0.90       197


Resultados con TF-IDF:
Accuracy: 0.9289340101522843
              precision    recall  f1-score   support

           0       0.95      0.92      0.94       112
           1       0.90      0.94      0.92        85

    accuracy                           0.93       197
   macro avg       0.93      0.93      0.93       197
weighted avg       0.93      0.93      0.93       197



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [55]:
# Your code
import scipy.sparse as sp

# Concatenar características adicionales con Bag of Words
X_bow_combined = sp.hstack([X_bow, data[['money_mark', 'suspicious_words', 'text_len']]])

# Concatenar características adicionales con TF-IDF
X_tfidf_combined = sp.hstack([X_tfidf, data[['money_mark', 'suspicious_words', 'text_len']]])

# Dividir los datos combinados en entrenamiento y prueba
X_train_bow_c, X_test_bow_c, y_train, y_test = train_test_split(X_bow_combined, data['label'], test_size=0.2, random_state=42)
X_train_tfidf_c, X_test_tfidf_c, _, _ = train_test_split(X_tfidf_combined, data['label'], test_size=0.2, random_state=42)

# Entrenar el modelo con Bag of Words combinado
model_bow_c = MultinomialNB()
model_bow_c.fit(X_train_bow_c, y_train)
y_pred_bow_c = model_bow_c.predict(X_test_bow_c)

# Entrenar el modelo con TF-IDF combinado
model_tfidf_c = MultinomialNB()
model_tfidf_c.fit(X_train_tfidf_c, y_train)
y_pred_tfidf_c = model_tfidf_c.predict(X_test_tfidf_c)

# Evaluar el modelo combinado
print("\nResultados con Bag of Words + Características adicionales:")
print("Accuracy:", accuracy_score(y_test, y_pred_bow_c))
print(classification_report(y_test, y_pred_bow_c))

print("\nResultados con TF-IDF + Características adicionales:")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf_c))
print(classification_report(y_test, y_pred_tfidf_c))



Resultados con Bag of Words + Características adicionales:
Accuracy: 0.8477157360406091
              precision    recall  f1-score   support

           0       1.00      0.73      0.85       112
           1       0.74      1.00      0.85        85

    accuracy                           0.85       197
   macro avg       0.87      0.87      0.85       197
weighted avg       0.89      0.85      0.85       197


Resultados con TF-IDF + Características adicionales:
Accuracy: 0.6142131979695431
              precision    recall  f1-score   support

           0       1.00      0.32      0.49       112
           1       0.53      1.00      0.69        85

    accuracy                           0.61       197
   macro avg       0.76      0.66      0.59       197
weighted avg       0.80      0.61      0.57       197



Final Thoughts
The results demonstrate the importance of combining traditional text-based feature representations (like BoW and TF-IDF) with domain-specific insights (e.g., suspicious words and monetary indicators). While TF-IDF provided a strong foundation, the additional features captured nuances of spam messages, leading to a robust and accurate SPAM/HAM classifier.

This approach highlights how domain knowledge and machine learning can work together to solve real-world text classification problems effectively.